# Mutation Analysis

In the [chapter on coverage](Coverage.ipynb), we showed how one identify which parts of the program are executed by a program, and hence get a sense of the effectiveness of a set of test cases in covering the program structure. However, is structural coverage a good measure of effectiveness? One of the problems with structural coverage measures is that it fails to check whether the program executions generated by the test suite were actually correct. That is, an execution that produces a wrong output that is unnoticed by the test suite is counted exactly the same as an execution that produces the right output for coverage. Indeed, if one deletes the assertions in a typical test case, the coverage would not change for the new test suite, but the new test suite is much less useful than the original one.

This is indeed, not an optimal state of affairs. How can we verify that our tests are actually useful? One alternative (hinted in the chapter on coverage) is to inject bugs into the program, and evaluate the effectiveness of test suites in catching these injected bugs. However, that that introduces another problem. How do we produce these bugs in the first place? Any manual effort is likely to be biased by the preconceptions of the developer as to where the bugs are likely to occur, and what effect it would have. Further, writing good bugs is likely to take a significant amount of time, for a very indirect benefit. Hence such a solution is not sufficient.  Mutation Analysis offers an alternative solution. The insight from Mutation Analysis is to consider the probability of insertion of a bug from the perspective of a programmer. If one assumes that the attention received by each program element in the program is sufficiently similar, one can further assume that each token in the program have a similar probability of being incorrectly transcribed. Of course, the programmer will correct any mistakes that gets detected by the compilers (or other static analysis tools). So the set of valid tokens different from the original that make it past the compilation stage is considered to be its possible set of _mutations_ that represent the _probable faults_ in the program. A test suite is then judged by its capability to detect (and hence prevent) such mutations. The proportion of such mutants detected over all _valid_ mutants produced is taken as the mutation score. In this chapter, we see how one can implement Mutation Analysis in Python programs. The mutation score obtained represents the ability of any program analysis tools to prevent faults, and can be used to judge static test suites, test generators such as fuzzers, and also static and symbolic execution frameworks.

**Prerequisites**

* You need some understanding of how a program is executed.
* You should have read [the chapter on coverage](Coverage.ipynb).

## _Section 1_

\todo{Add}

## _Section 2_

\todo{Add}

## _Section 3_

\todo{Add}

_If you want to introduce code, it is helpful to state the most important functions, as in:_

* `random.randrange(start, end)` - return a random number [`start`, `end`]
* `range(start, end)` - create a list with integers from `start` to `end`.  Typically used in iterations.
* `for elem in list: body` executes `body` in a loop with `elem` taking each value from `list`.
* `for i in range(start, end): body` executes `body` in a loop with `i` from `start` to `end` - 1.
* `chr(n)` - return a character with ASCII code `n`

In [ ]:
import fuzzingbook_utils

In [ ]:
def triangle(a, b, c):
    if a == b:
        if b == c:
            return 'Equilateral'
        else:
            return 'Isosceles'
    else:
        if b == c:
            return "Isosceles"
        else:
            if a == c:
                return "Isosceles"
            else:
                return "Scalene"

In [ ]:
import unittest

In [ ]:
class TestTriangle(unittest.TestCase):

    def test_equilateral(self):
        assert triangle(1,1,1) == 'Equilateral'

    def test_isosceles(self):
        assert triangle(1,2,1) == 'Isosceles'
        assert triangle(2,2,1) == 'Isosceles'
        assert triangle(1,2,2) == 'Isosceles'

    def test_scalene(self):
        assert triangle(1,2,3) == 'Scalene'

In [ ]:
def suite():
    suite = unittest.TestSuite()
    suite.addTest(TestTriangle('test_equilateral'))
    suite.addTest(TestTriangle('test_isosceles'))
    suite.addTest(TestTriangle('test_scalene'))
    return suite

def runTestMute():
    result = unittest.TestResult()
    suite().run(result)
    return result

def runTest():
    runner = unittest.TextTestRunner(verbosity=0, failfast=True)
    return runner.run(suite())

In [ ]:
runTest()

In [ ]:
class WeakTestTriangle(unittest.TestCase):
    def test_equilateral(self):
        assert triangle.triangle(1,1,1) == 'Equilateral'

    def test_isosceles(self):
        assert triangle.triangle(1,2,1) != 'Equilateral'
        assert triangle.triangle(2,2,1) != 'Equilateral'
        assert triangle.triangle(1,2,2) != 'Equilateral'

    def test_scalene(self):
        assert triangle.triangle(1,2,3) != 'Equilateral'

In [ ]:
runTest()

In [ ]:
import ast
import astunparse
import random

In [ ]:
class StmtCounter(ast.NodeVisitor):
    def __init__(self):
        self.i = 0

    def slightly_specific_visitor(self, node):
        self.i += 1
        return self.generic_visit(node)

    def visit_Return(self, node): return self.slightly_specific_visitor(node)
    def visit_Delete(self, node): return self.slightly_specific_visitor(node)

    def visit_Assign(self, node): return self.slightly_specific_visitor(node)
    def visit_AnnAssign(self, node): return self.slightly_specific_visitor(node)
    def visit_AugAssign(self, node): return self.slightly_specific_visitor(node)

    def visit_Raise(self, node): return self.slightly_specific_visitor(node)
    def visit_Assert(self, node): return self.slightly_specific_visitor(node)

    def visit_Global(self, node): return self.slightly_specific_visitor(node)
    def visit_Nonlocal(self, node): return self.slightly_specific_visitor(node)

    def visit_Expr(self, node): return self.slightly_specific_visitor(node)

    def visit_Pass(self, node): return self.slightly_specific_visitor(node)
    def visit_Break(self, node): return self.slightly_specific_visitor(node)
    def visit_Continue(self, node): return self.slightly_specific_visitor(node)

In [ ]:
def get_mutation_count(mysrc):
    s = StmtCounter()
    s.visit(ast.parse(mysrc))
    return s.i

In [ ]:
class StmtMutator(ast.NodeTransformer):
    def __init__(self, mutate_lst):
        self.i = 0
        self.mutate_lst = mutate_lst

    def slightly_specific_visitor(self, node):
        self.i += 1
        if self.i in self.mutate_lst:
            return ast.Pass()
        return self.generic_visit(node)

    def visit_Return(self, node): return self.slightly_specific_visitor(node)
    def visit_Delete(self, node): return self.slightly_specific_visitor(node)

    def visit_Assign(self, node): return self.slightly_specific_visitor(node)
    def visit_AnnAssign(self, node): return self.slightly_specific_visitor(node)
    def visit_AugAssign(self, node): return self.slightly_specific_visitor(node)

    def visit_Raise(self, node): return self.slightly_specific_visitor(node)
    def visit_Assert(self, node): return self.slightly_specific_visitor(node)

    def visit_Global(self, node): return self.slightly_specific_visitor(node)
    def visit_Nonlocal(self, node): return self.slightly_specific_visitor(node)

    def visit_Expr(self, node): return self.slightly_specific_visitor(node)

    def visit_Pass(self, node): return self.slightly_specific_visitor(node)
    def visit_Break(self, node): return self.slightly_specific_visitor(node)
    def visit_Continue(self, node): return self.slightly_specific_visitor(node)

In [ ]:
def parse(mysrc):
    s = StmtCounter()
    s.visit(ast.parse(mysrc))
    num_statements = s.i
    num_statements_to_mutate = num_statements
    mutate_lst = random.sample(range(1, num_statements+1), num_statements_to_mutate)
    v = StmtMutator(mutate_lst).visit(ast.parse(mysrc))
    return astunparse.unparse(parse(mysrc))

In [ ]:
def gen_mutant(mysrc, mutate_lst):
    v = StmtMutator(mutate_lst).visit(ast.parse(mysrc))
    return astunparse.unparse(v)

In [ ]:
import imp

In [ ]:
from ExpectError import ExpectTimeout

In [ ]:
def import_code(code, name):
    module = imp.new_module(name)
    exec(code, module.__dict__)
    return module

In [ ]:
def evalmutant(mname, mutant, test):
    test.__dict__[mname] = mutant
    with ExpectTimeout(1):
        v = test() # Was test run successful? False -- Mutant Found
    return not v.wasSuccessful()

In [ ]:
def mytest(lst_locations, fn_name, fn_src, test_code):
    try:
        mutant_src = gen_mutant(fn_src, lst_locations)
        mutant = import_code(mutant_src, fn_name)
        r = evalmutant(fn_name, mutant, test_code)
        return r
    except SyntaxError:
        print('Syntax!', lst_locations)
        return True

In [ ]:
def mutate(fn_name, fn_src, test_code):
    num_mutations = get_mutation_count(fn_src)
    mutate_lst = sorted(list(range(1, num_mutations+1)))
    res = []
    for m in mutate_lst:
        d = mytest([m], fn_name, fn_src, test_code)
        res.append(d)
    print(res)

In [ ]:
import inspect

In [ ]:
mutate('triangle', inspect.getsource(triangle), runTest)

## _Section 4_

\todo{Add}

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```markdown
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_